# Read and write images

In [1]:
import os
import bigfish
import bigfish.stack as stack
import bigfish.multistack as multistack
print("Big-FISH version: {0}".format(bigfish.__version__))

Big-FISH version: 0.6.2


In [2]:
# hard-code the paths of our input and output directories
# path_input = "../data/input"
# path_output = "../data/output"

path_input = "/data/smFISH/20251028_bartelle_smFISH_mm_microglia_newbuffers/data_r0003_tile0000_1"
path_output = "/home/mauri/big-fish-examples/notebooks"

# check input images are loaded
# stack.check_input_data(path_input)

This notebook shows examples to read and organize images. The output returned by a **Fluorescence In Situ Hybridization (FISH)** experiment consists in multichannel 2D or 3D images. For some techniques, several round of acquisitions compose the image. For these reasons, we expect to work with 5D tensors at the most: 
- one dimension r for the **round**
- one dimension c for the **channel**
- three **spatial** dimensions z,y and x

Yet, most of the analysis and transformations are applied at the channel level, directly over 3D or 2D images. For this reason, the beginning of the notebook show concrete examples to read and write single images.

## Read and write arrays

If you do not need to read different layers at the same time and rearrange them in a specific order, you can directly read files from their path:
- for *.png*, *.jpg*, *.jpeg*, *.tif* or *.tiff* extensions use `bigfish.stack.read_image`
- for *.dv* extension use `bigfish.stack.read_dv`

You can also read metadata or results specific file format:
- for *.npy* extension use `bigfish.stack.read_array`
- for *.npz* extension use `bigfish.stack.read_uncompressed`
- for *.csv* extension without header use `bigfish.stack.read_array_from_csv`
- for *.csv* extension with header use `bigfish.stack.read_dataframe_from_csv`

In [3]:
# path_dapi = os.path.join(path_input, "experiment_1_dapi_fov_1.tif")
image = stack.read_image("/data/smFISH/20251028_bartelle_smFISH_mm_microglia_newbuffers/data_r0003_tile0000_1/data_r0003_tile0000_NDTiffStack.tif", sanity_check = True)
print("\r shape: {0}".format(image.shape))
print("\r dtype: {0}".format(image.dtype))

# What is the shape format? z, y, x 

metadata = stack.read_dataframe_from_csv("/data/smFISH/20251028_bartelle_smFISH_mm_microglia_newbuffers/scan_metadata.csv")

 shape: (189, 2048, 2048)
 dtype: uint16


Same logic if you want to save images or arrays:
- for *.png*, *.jpg*, *.jpeg*, *.tif* or *.tiff* extensions use `bigfish.stack.save_image`
- for *.npy* extension use `bigfish.stack.save_array`
- for *.csv* extension use `bigfish.stack.save_data_to_csv`

## Experiences, fields of view and recipes

We assume a biologist to acquire one or several **fields of view (FoV)** per **experiment**. All images from the same experiment should be in the same input directory.

To load these fov with bigFISH, we need to directly read the 3D or 2D images and organize them by channels and rounds if necessary. For this purpose, **we define recipes that map together the different layers of a FoV**. We also expect the biologist to directly save the 3D or 2D images, with an appropriate name including the information about the experiment, the fov, the channel and the round.

In [ ]:
# checks what files are in your input path
os.listdir(path_input)

['NDTiff.index', 'data_r0003_tile0000_NDTiffStack.tif']

For example, in our input directory we saved two files corresponding to two different channels (dapi and FISH) from the same fov and the same experiment. Each file is a 3D image. We want to read these two images and stack them together along a new channel dimension. A correct recipe would be:

In [ ]:
recipe = {
    "fov": "fov_1",
    "c": ["aif1", "gene2"],
    "opt": "experiment_1",
    "ext": "tif",
    "pattern": "opt_c_fov.ext"}

# This is a helpful function if your files follow the same naming scheme. 
# It assumes your channels are in separate files. What will it do if my 2 channels are in 1 file?
# data_r0003_tile0000_NDTiffStack.tif

In this recipe, we specify different keys:
- the **FoV** (*fov_1*) shared by the two files
- an **optional term** (*experiment_1*) shared by the two files
- a list of keys to distinguish the different **channels** (*dapi* and *smfish*)
- the **extension** name (*tif*) shared by the two files
- the **general pattern** of the filenames (*opt_c_fov.ext* as a template for *experiment_1_{c}\_fov_1.tif*)

We can use the elements provided in the recipe to recognize the filenames of the different files, read them and gather them accordingly in order to build a 5D image: **(round, channel, z, y, x)**.

In [6]:
# function to check if your recipe is correctly defined and files actually exist
multistack.check_recipe(recipe, data_directory=path_input)

FileNotFoundError: File does not exist: /data/smFISH/20251028_bartelle_smFISH_mm_microglia_newbuffers/data_r0003_tile0000_1/experiment_1_dapi_fov_1.tif

__Note 1:__ A recipe can take into account others elements. The general pattern excepted, these elements are all optional and depend of the filenames we want to recognize. If a 3D image is decomposed in 2D layers, a __z__ key would be necessary in order to correctly stack the 2D images along the z axis (in the right order). The same logic happens if we want to stack layers from different rounds to build our final image, with a **r** key.

    {
    "fov": str or List[str], (optional)
    "z": str or List[str],   (optional)
    "c": str or List[str],   (optional)
    "r": str or List[str],   (optional)
    "ext": str,              (optional)
    "opt": str,              (optional)
    "pattern": str
    }
    
__Note 2:__ If you have a large number of images to read, keeping the same template to name your files will simplify your life.

### Build a multichannel image with a recipe

In [ ]:
image = multistack.build_stack(recipe, input_folder=path_input) 
print("\r shape: {0}".format(image.shape))
print("\r dtype: {0}".format(image.dtype))

 shape: (1, 2, 23, 650, 500)
 dtype: uint16


As expected, we return a 3D image with 1 round and 2 channels. 

### Build several multichannel images with recipes

If we want to read several multichannel images, we can define a **generator** (files are read and image is build only when it is called). The first step consists in **mapping different recipes with their relative input directory**:

In [ ]:
# as an example, we just duplicate the same recipe
data_map = [
    (recipe, path_input),
    (recipe, path_input),
    (recipe, path_input),
]

# function to check if your mapping (recipe, input directory) is correct
multistack.check_datamap(data_map)

True

In [ ]:
image_generator = multistack.build_stacks(data_map)
for i, image in enumerate(image_generator):
    print("Image {0}".format(i))
    print("\r shape: {0}".format(image.shape))
    print("\r dtype: {0}".format(image.dtype), "\n")

Image 0
 shape: (1, 2, 23, 650, 500)
 dtype: uint16 

Image 1
 shape: (1, 2, 23, 650, 500)
 dtype: uint16 

Image 2
 shape: (1, 2, 23, 650, 500)
 dtype: uint16 



__Note:__ A generator can be used to read several images from the same recipe if you provide several FoV in this recipe.

In [ ]:
recipe = {
    "fov": ["fov_1", "fov_1"],
    "c": ["dapi", "smfish"],
    "opt": "experiment_1",
    "ext": "tif",
    "pattern": "opt_c_fov.ext"}
data_map = [(recipe, path_input)]
image_generator = multistack.build_stacks(data_map)
for i, image in enumerate(image_generator):
    print("Image {0}".format(i))
    print("\r shape: {0}".format(image.shape))
    print("\r dtype: {0}".format(image.dtype), "\n")

Image 0
 shape: (1, 2, 23, 650, 500)
 dtype: uint16 

Image 1
 shape: (1, 2, 23, 650, 500)
 dtype: uint16 



### Build a multichannel image without recipe

To load an image without defining a recipe, we need to list the **paths** of the different layers to stack together, in the right order.

In [ ]:
# list paths. of the files to stack
path_dapi = os.path.join(path_input, "experiment_1_dapi_fov_1.tif")
path_smfish = os.path.join(path_input, "experiment_1_smfish_fov_1.tif")
paths = [path_dapi, path_smfish]

# load the layers and build the image
image = multistack.build_stack_no_recipe(paths)
print("\r shape: {0}".format(image.shape))
print("\r dtype: {0}".format(image.dtype))

 shape: (1, 2, 23, 650, 500)
 dtype: uint16
